# Algoritmo de Shor

## Ilustração das principais ideias

Em criptografia, usa-se frequentemente o problema de fatoração de números inteiros como o produto de números primos. Embora o problema de multiplicação de inteiros seja resolvido rapidamente em computadores clássicos, e portanto podemos facilmente obter
$$N=a\times b$$
conhecendo $a$ e $b$, o problema inverso, de dado
$$N=?\times ?$$
obter os fatores primos, é muito difícil para um computador clássico.

O melhor algoritmo clássico conhecido atualmente para resolver esse problema é o Number Field Sieve (algoritmo de peneira de campo) (NFS). No entanto, esse algoritmo levaria $2000$ anos em um processador clássico para fator números inteiros usados no protocolo de criptografia conhecido como RSA-$768$. Não existe prova da inexistência de algoritmos clássicos mais eficientes. Não obstante, já sabemos da existência de uma algoritmo quântico, o algoritmo de Shor, que resolve o problema de fatoração de forma eficiente.

Os melhores algoritmos clássicos resolvem o problema de fatoração chutando aleatoriamente candidatos a fatores primos e checando se são ou não. Como existem muitos números para checar, esses algoritmos são ineficientes. No entanto, vale observar que não precisamos obter necessariamente um fator de $N$, mas sim um número $g$ que compartilhe um fator com $N$. Isso porque existe um algoritmo clássico, o algoritmo de Euclides, que é eficiente para calcular o maior divisor comum
$$\gcd(g,N).$$

Digamos então que começamos com um palpite $g$ para o fator de $N$. É muito improvável que $g$ e $N$ compartilhem algum fator. No entanto, existe um resultado matemático que estabelece que se dois números $A$ e $B$ não compartilham nenhum fator além de $1$ (são coprimos), então se multiplicamos $A$ por ele mesmo um certo número $p$ de vezes, então poderemos escrever
$$A^p = m\times B + 1,$$
com $p$ e $m$ sendo um números naturais.

Comsideremos alguns exemplos. Os números $7$ e $15$ não compartilham nenhum fator. Mas
\begin{align}
& 7^2 = 49 = 3\times 15 + 3,\\
& 7^3 = 343 = 22\times 15 + 13,\\
& 7^4 = 2401 = 160\times 15 + 1.
\end{align}
Os números $42$ e $13$ também não compartilham nenhum fator. Mas
\begin{align}
& 42^2 = 1764 = 135\times 13 + 9,\\
& 42^3 = 74088 = 5699\times 13 + 1.
\end{align}

In [ ]:
42**2, 135*13, 42**3, 5699*13

(1764, 1755, 74088, 74087)

Então, para o nosso __palpite errado__ $g$, existem números naturais $p$ e $m$ tais que
\begin{align}
& g^p = m\times N + 1 \\
& \therefore\ g^p - 1 = m\times N \\
& \therefore\ \big(g^{p/2} + 1\big)\big(g^{p/2} - 1\big) = m\times N.
\end{align}
Os números
$$g^{p/2} \pm 1$$
são __melhores candidatos__ do que $g$ para serem fatores de $N$.

No entanto, podem ocorrer alguns problemas:

1. Um dos valores $g^{p/2} \pm 1$ pode ser ele mesmo um múltiplo de $N$ e/ou de $m$:
$$g^{p/2} \pm 1 = t\times N,\ g^{p/2} \pm 1 =  w\times m,$$
com $t$ e $w$ sendo números naturais. Nestes casos, $g^{p/2} \pm 1$ não seriam úteis. No entanto, podemos verificar eficientemente se esses casos ocorrem usando o algoritmo de Euclides.

2. O numero $p$ pode ser ímpar. Nesse caso $p/2$ não é inteiro, e portanto $g^{p/2}\pm 1$ também não é inteiro. Por outro lado, sabe-se que $g^{p/2} \pm 1$ compartilhará um fator com $N$ para aproximadamente $37.5\ \%$ (com probabilidade $3/8$) dos chutes iniciais $g$. Com isso, vemos que para encontrar um valor $g^{p/2} \pm 1$ que compartilha um fator com $N$ com probabilidade maior que
$$0.99,$$
precisamos de não mais que
$$10\text{ chutes}.$$

3. Não conhecemos o valor de $p$. O problema de encontrar $p$ em um computador clássico é tão ou mais difícil que o problema de fatoração. Por outro lado, computadores quânticos nos ajudam a obter $p$ muito rapidamente.

O segredo da computação quântica está em calcular a função em diferentes valores do domínio em superposição:
$$|a\rangle+|b\rangle+|c\rangle+\cdots \xrightarrow{f(x)}|f(a)\rangle+|f(b)\rangle+|f(c)\rangle+\cdots \xrightarrow{\text{interferência}}|\text{resposta}\rangle.$$

É difícil fazer isso, mas Shor fez. E podemos usar o algoritmo de Shor para encontrar $p$ tal que $g^p = m\times N+1$. Encontrando $p$, temos que $g^{p/2}\pm 1$ tem alta probabilidade de compartilhar um fator com $N$.

Precisaremos de uma sequência de operações do tipo
$$|x\rangle \xrightarrow{g^x} |x,g^x\rangle \xrightarrow{>m\times N} |x,+r\rangle,$$
em que $+r$ representa o quão maior que $1$ é o resto da divisão de $g^x$ por $N$, algo to tipo
\begin{align}
& |1\rangle+|2\rangle+|3\rangle+\cdots \xrightarrow{g^x} |1,g^1\rangle+|2,g^2\rangle+|3,g^3\rangle+\cdots \\
& \xrightarrow{>m\times N} |x,+17\rangle+|x,+5\rangle+ |x,+92\rangle+\cdots.
\end{align}
Nesse ponto, não podemos simplesmente medir na base computacional, pois obteríamos um único elemento da superposição. Precisamos fazer uma operação esperta que cancela os termos que não queremos, restando somente a resposta
$$|p,+1\rangle.$$

Aqui é importante nota que se
$$g^x = m\times N + r$$
então, por exemplo, teremos que
\begin{align}
& g^{42} = m_1\times N+3 \\
& \therefore\ g^{42+p} = g^{42}g^p = (m_1\times N+3)(m\times N+1) \\
& = m_1 m N\times N + m_1\times N + 3m\times N + 3 \\
& = (m_1 m N + m_1 + 3m)\times N + 3 \\
& = m_3\times N + 3,
\end{align}
com $m_3\in \mathbb{N}.$ Do mesmo modo, podemos mostrar que
$$g^{42 + t\times p} = m_t\times N + 3,$$
com $t,m_t\in\mathbb{N}.$ Claro, esse resultado não vale somente para $42$ e para o resto $3$, vale para qualquer número natural $x$ e para qualquer resto $r$:
$$g^{x + t\times p} = m_t\times N + r.$$

Ou seja, o valor da potência $p$ que procuramos tem a __propriedade periódica__ que $x$ e $x+t\times p$ retornam o mesmo valor $r$ para o resto da divisão de $g^{x+t\times p}$ por $N$:
$$g^{x + 0\times p} = m_0\times N + r\ \therefore\ g^{x + t\times p} = m_t\times N + r.$$
Essa propriedade de periodicidade é uma relação entre as diferentes potências.

Assim, no computador quântico, se preparamos uma superposição de todas as potências, com e.g.
$$|1,+17\rangle + |2,+3\rangle + |3,+92\rangle + \cdots$$
e medimos o 2º registro obtendo e.g. o estado
$$|+3\rangle,$$
deixamos o sistema e.g. no seguinte estado de superposição
$$|2,+3\rangle + |12,+3\rangle + |22,+3\rangle + \cdots.$$

Vale lembrar que esse efeito é análogo ao que tínhamos no algoritmo de Simon.

Assim, o estado do 1º registro é a superposição
\begin{align}
& |2\rangle + |12\rangle + |22\rangle + \cdots = |2+0\times 10\rangle + |2+1\times 10\rangle + |2+2\times 10\rangle + |2+0\times 10\rangle \\
& = \sum_{j=0}^{M} |2+j\times p\rangle.
\end{align}
Note que este estado tem periodicidade de frequência
$$f = \frac{1}{\text{período}} = \frac{1}{p}.$$

A melhor ferramenta conhecida para encontrar frequência é a transformada de Fourier. Em particular, usando a transformada de Fourier quântica, teremos que
$$\sum_{j=0}^{M} |x+j\times p\rangle \xrightarrow{TFQ} \Big|\frac{1}{p}\Big\rangle.$$

Com isso, finalizamos a descrição sucinta das principais ideias envolvidas no algoritmo de fatoração de Shor, a saber
\begin{align}
& \frac{1}{1/p} \rightarrow p \xrightarrow{\text{se par}} g^{p/2}\pm 1 \\
& \xrightarrow[]{\text{se não é múltiplo de }N} \text{compartilha fator com }N \\
& \rightarrow \text{Usa o algoritmo de Euclides} \\
& \rightarrow \text{Encontra os fatores tais que:} N=a\times b.
\end{align}

## Referências

1. P. W. Shor, “Algorithms for quantum computation: discrete logarithms and factoring,” in Proceedings 35th Annual Symposium on Foundations of Computer Science, Nov. 1994, pp. 124-134. doi: 10.1109/SFCS.1994.365700.

1. P. W. Shor, “Polynomial-Time Algorithms for Prime Factorization and Discrete Logarithms on a Quantum Computer,” SIAM J. Comput., vol. 26, no. 5, pp. 1484-1509, Oct. 1997, doi: 10.1137/S0097539795293172.

1. M. A. Nielsen and I. L. Chuang, Quantum Computation and Quantum Information (Cambridge University Press,Cambridge, 2000).

2. S. Aaronson, The Prime Facts: From Euclid to AKS, https://www.scottaaronson.com/writings/prime.pdf (2003).

1. M. A. Nielsen and I. L. Chuang, Quantum Computation and Quantum Information (Cambridge University Press, Cambridge, England, 2000).

1. Richard Cleve, Introduction to Quantum Information Processing (2023), https://cs.uwaterloo.ca/~cleve/courses/F11CS667/, https://youtube.com/playlist?list=PLgOc9DWm_Ey3cnHXjmu8f__ki5AJYd53p&si=etCdYsFeCUBTgclj.

1. Dave Bacon, Lecture Notes on Quantum Computing (2006), https://courses.cs.washington.edu/courses/cse599d/06wi/.

1. John Preskill, Lecture Notes on Quantum Computation, http://theory.caltech.edu/~preskill/ph219/index.html#lecture.

1. A. Javadi-Abhari et al., “Quantum computing with Qiskit.” arXiv, May 14, 2024. doi: 10.48550/arXiv.2405.08810.

1. minutephysics, How Quantum Computers Break Encryption - Shor's Algorithm Explained, https://youtu.be/lvTqbM5Dq4Q?si=jBT1wgktbrgmDtJ2.

1. minutephysics, How Shor's Algorithm Factors 314191, https://youtu.be/FRZQ-efABeQ?si=pKsbGrK25GkfJyxa.
